# Report

In [1]:
%cd ..
%pwd

/Users/andersvandvik/Repositories/project-thesis


'/Users/andersvandvik/Repositories/project-thesis'

In [3]:
import data
import folium
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pprint import pprint
%matplotlib inline

In [10]:
# TODO: This must be changed
inst_to_coord = {
    0: [60.8024, 5.0107],
    1: [60.64, 3.72],
    2: [60.77, 3.50],
    3: [60.88, 3.60],
    4: [60.74, 3.61],
    5: [60.51, 3.26],
    6: [60.95, 3.58],
    7: [60.89, 3.67],
    8: [60.90, 3.81],
    9: [60.08, 2.63],
    10: [60.48, 2.82],
    11: [60.48, 2.82],
    12: [60.60, 2.77],
    13: [60.70, 2.93],
    14: [60.70, 2.93],
    15: [60.38, 2.79],
    16: [61.07, 2.50],
    17: [61.04, 2.34],
    18: [61.04, 2.34],
    19: [60.78, 2.89],
    20: [60.85, 2.65],
    21: [60.85, 2.65],
    22: [61.25, 1.85],
    23: [61.20, 1.82],
    24: [61.29, 1.90],
    25: [61.17, 2.18],
    26: [61.20, 2.20],
    27: [61.20, 2.27]
}

vessel_to_color = {
    0: 'blue',
    1: 'green',
    2: 'yellow',
    3: 'violet',
    4: 'orange',
    5: 'red'
}

In [125]:
def summarize_voyages(voyages):
    column_names = ['Vessel', 'Start node', 'End node', 'Start time', 
                  'Arrival time', 'End time', 'Delivery load', 'Pickup load',
                  'Arc fuel cost', 'Arc charter cost']
    df = pd.DataFrame(columns=column_names)
    for vessel in voyages.keys():
        for start_node in voyages[vessel]:
            leg_info = voyages[vessel][start_node]
            end_node = leg_info[0]
            start_time, arr_time, end_time = leg_info[1]
            delivery_load, pickup_load = leg_info[2]
            arc_fuel_cost, arc_charter_cost = leg_info[3]
            row = [int(vessel), int(start_node), end_node, start_time, arr_time, end_time, 
                   float(delivery_load), float(pickup_load), arc_fuel_cost, arc_charter_cost]
            row_df = pd.DataFrame([row], columns=column_names)
            df = df.append(row_df)
    return df

def organize_solution_attributes(postponed_orders, serviced_orders, fuel_costs, charter_costs, 
                                 penalty_costs, model_runtime, preprocess_runtime):
    index_names = ['Postponed orders', 'Serviced orders', 
                   'Fuel costs', 'Charter costs', 'Penalty costs', 
                   'Model runtime', 'Preprocess runtime']
    column_names = ['Value']
    df = pd.DataFrame([[postponed_orders], [serviced_orders], [fuel_costs], [charter_costs], 
                       [penalty_costs], [model_runtime], [preprocess_runtime]], 
                      columns=column_names, index=index_names)
    return df

def organize_instance_info(fleet_size, inst_ordering, number_of_insts, weather_scenario, number_of_orders):
    index_names = ['Installations', 'Number of orders', 'Fleet_size', 'Installation ordering', 'Weather scenario']
    column_names = ['Value']
    df = pd.DataFrame([[number_of_insts], [number_of_orders], [fleet_size], [inst_ordering], 
                       [weather_scenario]], columns=column_names, index=index_names)
    return df

def organize_order_composition(order_composition):
    column_names = ['Order', 'Size', 'Installation']
    df = pd.DataFrame(columns=column_names)
    for order in order_composition.keys():
        order_type = order_composition[order]['order']
        size = order_composition[order]['size']
        installation = order_composition[order]['installation']
        row = [order_type, size, installation]
        row_df = pd.DataFrame([row], columns=column_names)
        df = df.append(row_df)
    return df

def define_map():
    middle = [60.793142, 3.601824]
    m = folium.Map(location=middle,
                   zoom_start=8,
                   zoom_control=False)
    folium.TileLayer('cartodbpositron').add_to(m)
    return m

def add_markers_and_legs(sequence, m, vessel_to_color, inst_to_coord, vessel):
    for leg in sequence:
        print(leg)
        inst = data.ALL_NODES[leg[0]].get_installation()
        color = vessel_to_color[vessel]
        folium.Marker(location=inst_to_coord[inst.get_index()], fill_color=color, radius=3).add_to(m)        
    for leg in sequence:
        start_inst, end_inst = data.ALL_NODES[leg[0]].get_installation(), data.ALL_NODES[leg[2]].get_installation()
        color = vessel_to_color[vessel]
        folium.PolyLine(locations=[inst_to_coord[start_inst.get_index()], 
                                   inst_to_coord[end_inst.get_index()]], 
                                   color=color, weight=2).add_to(m)

In [132]:
file_path = '/Users/andersvandvik/Repositories/project-thesis/output/results/C-I3-V3-WS2.json'

with open(file_path) as results:
    json_file = json.load(results)
    
voyages = json_file['voyages']

model_runtime = json_file['runtime']['model_runtime']
preprocess_runtime = json_file['runtime']['preprocess_runtime']

fuel_costs = json_file['objective']['fuel_costs']
charter_costs = json_file['objective']['charter_costs']
penalty_costs = json_file['objective']['penalty_costs']

postponed_orders = json_file['order_fulfillment']['postponed_orders']
serviced_orders = json_file['order_fulfillment']['serviced_orders']
number_of_orders = len(postponed_orders) + len(serviced_orders)

fleet_size = json_file['instance_info']['fleet_size']
inst_ordering = json_file['instance_info']['installation_ordering']
number_of_insts = json_file['instance_info']['number_of_installations']
order_composition = json_file['instance_info']['order_composition']
weather_scenario = json_file['instance_info']['weather_scenario']

In [127]:
instance_info = organize_instance_info(fleet_size, inst_ordering, number_of_insts, 
                                       weather_scenario, number_of_orders)
voyage_summary = summarize_voyages(voyages)
solution_info = organize_solution_attributes(postponed_orders, serviced_orders, fuel_costs, 
                                             charter_costs, penalty_costs, model_runtime, preprocess_runtime)
order_composition = organize_order_composition(order_composition)

In [128]:
instance_info

,Value
Installations,3
Number of orders,5
Fleet_size,3
Installation ordering,Clustered
Weather scenario,2


In [129]:
order_composition

,Order,Size,Installation
0,MD_SOD,18.75,27
0,OD_SOD,11.25,27
0,OD_SEQ,25.00,7
0,MD_TRO,15.00,1
0,OP_TRO,15.00,1


In [130]:
voyage_summary

,Vessel,Start node,End node,Start time,Arrival time,End time,Delivery load,Pickup load,Arc fuel cost,Arc charter cost
0,0,0,6,63,63,63,0.00,0.0,0.000000,0.0
0,1,0,1,63,80,88,70.00,0.0,454.587336,0.0
0,1,1,2,88,88,93,51.25,0.0,58.543750,0.0
0,1,2,3,93,95,105,40.00,0.0,179.603131,0.0
0,1,3,4,105,112,130,15.00,0.0,350.016154,0.0
0,1,4,5,130,130,136,0.00,0.0,84.303000,0.0
0,1,5,6,136,155,155,0.00,15.0,457.915764,0.0
0,2,0,6,63,63,63,0.00,0.0,0.000000,0.0
0,3,0,6,63,63,63,0.00,0.0,0.000000,0.0


In [131]:
solution_info

,Value
Postponed orders,[]
Serviced orders,"[1, 2, 3, 4, 5]"
Fuel costs,1584.97
Charter costs,0
Penalty costs,4.54747e-13
Model runtime,0.643698
Preprocess runtime,2.19958


In [32]:


m = define_map()

for vessel in range(fleet_size + 1):
    add_markers_and_legs(vessel_sequences[str(vessel)], m, vessel_to_color, inst_to_coord, vessel)

# TODO: Map order number to installation by adding installation to order_combinations    

m

NameError: name 'vessel_sequences' is not defined

## Route

## Speed profile

In [ ]:
fig = go.Figure()

X, Y = [], []
for vessel in range(len(data.VESSELS)):
    speeds = []
    time_points = []
    for sequence in json_file[str(vessel)]:
        start_node, start_time, end_node, end_time, d_load, p_load, speed = sequence
        if speed == 0:
            continue
        start_inst, end_inst = data.ALL_NODES[start_node].get_installation(), data.ALL_NODES[end_node].get_installation()
        print(f'{start_inst.get_name()} -> {end_inst.get_name()}: {speed}')
        X.append([tp for tp in range(start_time, end_time)])
        Y.append([speed] * (end_time - start_time))
    
    for x, y in zip(X, Y):
        plt.plot(x, y, color='b')
        
    for x, y in zip(X, Y):
        fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='royalblue', width=2)))

flat_X = [x_coord for sub_x in X for x_coord in sub_x]
lowest_speed = [7 for _ in range(len(flat_X))]
eight_speed = [8 for _ in range(len(flat_X))]
nine_speed = [9 for _ in range(len(flat_X))]
ten_speed = [10 for _ in range(len(flat_X))]
eleven_speed = [11 for _ in range(len(flat_X))]
twelve_speed = [12 for _ in range(len(flat_X))]
thirteen_speed = [13 for _ in range(len(flat_X))]
highest_speed = [14 for _ in range(len(flat_X))]
plt.plot(flat_X, efficient_speed)
plt.plot(flat_X, lowest_speed)

fig.add_trace(go.Scatter(x=flat_X, y=lowest_speed, mode='lines', line=dict(color='firebrick', width=1)))
fig.add_trace(go.Scatter(x=flat_X, y=eight_speed, mode='lines', line=dict(color='tan', width=1, dash='dot')))
fig.add_trace(go.Scatter(x=flat_X, y=nine_speed, mode='lines', line=dict(color='tan', width=1, dash='dot')))
fig.add_trace(go.Scatter(x=flat_X, y=ten_speed, mode='lines', line=dict(color='tan', width=1, dash='dot')))
fig.add_trace(go.Scatter(x=flat_X, y=eleven_speed, mode='lines', line=dict(color='tan', width=1, dash='dot')))
fig.add_trace(go.Scatter(x=flat_X, y=twelve_speed, mode='lines', line=dict(color='tan', width=1, dash='dot')))
fig.add_trace(go.Scatter(x=flat_X, y=thirteen_speed, mode='lines', line=dict(color='tan', width=1, dash='dot')))
fig.add_trace(go.Scatter(x=flat_X, y=highest_speed, mode='lines', line=dict(color='firebrick', width=1)))


fig.update_layout(yaxis=dict(
    showgrid=True, 
    zeroline=True, 
    showline=True, 
    showticklabels=True), 
                  plot_bgcolor='white')

plt.show()
fig.show()